In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import plotly.express as px
import plotly.graph_objs as go
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
init_notebook_mode(connected=True)

#### 1. Read the initial dataset

In [ ]:
data = pd.read_csv('data/lipid_data.csv', low_memory=False)
metadata = pd.read_csv('data/metadata.csv', low_memory=False)

In [ ]:
mz = data['m/z']
adducts = [str(d) for d in data['Adducts']]
mz2adducts = dict(zip(mz, adducts))

#### 2. Read the normalized dataset from Normalization-step

In [ ]:
norm_data = pd.read_csv('data/normalized_data.csv')
tissue_matchings = {'plasma': 'plasma', 'Liver':'liver', 'Muscle':'muscle',
                    'Brain (CB)': 'brain_cb', 'Brain (PFC)': 'brain_pfc'}
norm_data['tissue'] = norm_data.Tissue.map(tissue_matchings)
norm_data['age'] = norm_data.rat_type.map({'young no': 'young', 'old no':'old', 'old yes':'old'})
norm_data['vitamin'] = norm_data.rat_type.map({'young no': 'no', 'old no':'no', 'old yes':'yes'})
norm_data['sample'] = norm_data['Unnamed: 0']

#### 3. Select top 1000 m/z peaks for each tissue

In [ ]:
MZ_COLUMNS = sorted(list(set(norm_data.columns) - set(
    ['Unnamed: 0', 'tissue', 'age', 'rat_type', 'vitamin', 'sample', 'Tissue'])))

In [ ]:
def select_top_mz(df, tissue, n):
    max_mz = list(df[df.tissue == tissue][MZ_COLUMNS].mean().sort_values(ascending=False).index)[0:n]
    return df[df.tissue == tissue][max_mz + ['sample', 'age', 'vitamin']]

tissues_top = {}
for tissue in ['plasma', 'liver', 'muscle', 'brain_cb', 'brain_pfc']:
    tissues_top[tissue] = select_top_mz(norm_data, tissue, 1000)

In [ ]:
tissues_top['liver'].head()

#### 4. Calculate basic m/z statistics

For every group of rats (old, young and old with vitamin D) for each m/z peak calculate median value of all samples

In [ ]:
def calc_mz_stat(df):
    mz_columns = sorted(list(set(df.columns) - set(
    ['Unnamed: 0', 'tissue', 'age', 'rat_type', 'vitamin', 'sample', 'Tissue'])))
    young = df[(df.age == 'young')]
    old = df[(df.age == 'old')&(df.vitamin == 'no')]
    old_d = df[(df.age == 'old')&(df.vitamin == 'yes')]
    new_df = pd.DataFrame(young[mz_columns].median().reset_index())
    new_df.columns = ['mz', 'median_young']
    new_df['median_old'] = old[mz_columns].median().values
    new_df['median_old_d'] = old_d[mz_columns].median().values
    new_df['old_young_diff'] = (new_df.median_old - new_df.median_young) / new_df.median_young
    new_df['old_d_young_diff'] = (new_df.median_old_d - new_df.median_young) / new_df.median_young
    new_df['old_d_old_diff'] = (new_df.median_old_d - new_df.median_old) / new_df.median_old
    new_df['abs_old_young_diff'] = np.abs(new_df.median_old - new_df.median_young) / new_df.median_young
    new_df['adducts'] = new_df.mz.map(lambda x: mz2adducts[float(x)])
    new_df = new_df[(new_df.median_old > 0)&(new_df.median_old_d > 0)&(new_df.median_young > 0)]
    return new_df.sort_values('abs_old_young_diff', ascending=False)
    
tissue_2_mz_stat = {}
for tissue in ['plasma', 'liver', 'muscle', 'brain_cb', 'brain_pfc']:
    tissue_2_mz_stat[tissue] = calc_mz_stat(tissues_top[tissue])

Calculate average diff between medians of peaks of old rats and old rats with vitamin D (in all tissues separetely)

In [ ]:
print("Tissue", "N peaks", "Old", "Old+D")
for tissue in ['plasma', 'liver', 'muscle', 'brain_cb', 'brain_pfc']:
    tmp = tissue_2_mz_stat[tissue]
    print(tissue, tmp.shape[0], tmp.old_young_diff.mean(), tmp.old_d_young_diff.mean())

Colorize positive and negative deviations

In [ ]:
def color_map(value):
    color = 'black'
    if type(value) == type(0.1):
        if value > 1:
            color = 'green'
        elif value < -0.5:
            color = 'red'
    return 'color: %s' % color

In [ ]:
tissue_2_mz_stat['plasma'].sort_values('mz').style.applymap(color_map)

In [ ]:
tissue_2_mz_stat['liver'].sort_values('mz').style.applymap(color_map)

In [ ]:
tissue_2_mz_stat['muscle'].sort_values('mz').style.applymap(color_map)

In [ ]:
tissue_2_mz_stat['brain_cb'].style.applymap(color_map)

In [ ]:
tissue_2_mz_stat['brain_pfc'].style.applymap(color_map)

#### 5. Draw volcano-plots

In [ ]:
def calc_volcano(df_etalon, df_test):
    mz_columns = sorted(list(set(df_etalon.columns) - set(
    ['Unnamed: 0', 'tissue', 'age', 'rat_type', 'vitamin', 'sample', 'Tissue'])))
    volcano_plot_df = pd.DataFrame(df_etalon[mz_columns].median().reset_index())
    volcano_plot_df.columns = ['mz', 'etalon_median']
    volcano_plot_df['test_median'] = df_test[mz_columns].median().values
    volcano_plot_df['text'] = volcano_plot_df.mz + '_' + volcano_plot_df.mz.map(lambda x: mz2adducts[float(x)])
    volcano_plot_df = volcano_plot_df[(volcano_plot_df.test_median > 0)&(volcano_plot_df.etalon_median > 0)]
    volcano_plot_df['FC'] = volcano_plot_df.test_median/volcano_plot_df.etalon_median
    volcano_plot_df['log2FC'] = np.log2(volcano_plot_df.FC)
    anova_pvalue = []
    for mz in volcano_plot_df['mz'].values:
        anova_pvalue.append(scipy.stats.f_oneway(df_etalon[mz].values, df_test[mz].values).pvalue)
    volcano_plot_df['pvalue'] = anova_pvalue
    volcano_plot_df['minus_log10_pvalue'] = np.log10(anova_pvalue) * -1
    return volcano_plot_df

def volcano_color(log2fc):
    if log2fc >= 2:
        return 'green'
    if log2fc <= -2:
        return 'red'
    return 'grey'

def volcano_figures(df1, df2, title, max_val):
    traces1 = []
    traces1.append(go.Scatter(x=df1.log2FC, y=df1.minus_log10_pvalue,
                              mode='markers',
                              marker=dict(color=[volcano_color(lfc) for lfc in df1.log2FC.values]),
                              name='m/z points',
                              text=df1.text.values))
    traces1.append(go.Scatter(x=df1.log2FC,
                              y=[-np.log10(0.05)] * df1.log2FC.shape[0],
                              line=dict(color='red', dash='dash'),
                              name='p_value=0.05'))
    traces1.append(go.Scatter(x=df1.log2FC,
                              y=[max_val] * df1.log2FC.shape[0],
                              line=dict(color='white'),
                              name='y=3.5'))
    
    traces2 = []
    traces2.append(go.Scatter(x=df2.log2FC, y=df2.minus_log10_pvalue,
                              mode='markers',
                              marker=dict(color=[volcano_color(lfc) for lfc in df2.log2FC.values]),
                              name='m/z points',
                              text=df2.text.values))
    traces2.append(go.Scatter(x=df2.log2FC,
                              y=[-np.log10(0.05)] * df2.log2FC.shape[0],
                              line=dict(color='red', dash='dash'),
                              name='p_value=0.05'))
    traces2.append(go.Scatter(x=df2.log2FC,
                              y=[max_val] * df2.log2FC.shape[0],
                              line=dict(color='white'),
                              name='y=3.5'))
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Old/Young", "OldD/Young"))
    fig.add_trace(traces1[0], row=1, col=1)
    fig.add_trace(traces1[1], row=1, col=1)
    fig.add_trace(traces1[2], row=1, col=1)
    fig.add_trace(traces2[0], row=1, col=2)
    fig.add_trace(traces2[1], row=1, col=2)
    fig.add_trace(traces2[2], row=1, col=2)
    fig.update_layout(height=600, width=1000, title_text=title)
    fig.update_xaxes(title_text="log2 fold change", row=1, col=1)
    fig.update_xaxes(title_text="log2 fold change", row=1, col=2)
    fig.update_yaxes(title_text="-log10(pvalue)", row=1, col=1)
    fig.update_yaxes(title_text="-log10(pvalue)", row=1, col=2)
    return fig

def volcano_figure(df, title):
    traces = []
    traces.append(go.Scatter(x=df.log2FC, y=df.minus_log10_pvalue,
                             mode='markers',
                             marker=dict(color=[volcano_color(lfc) for lfc in df.log2FC.values]),
                             name='m/z points',
                             text=df.text.values))
    traces.append(go.Scatter(x=df.log2FC,
                             y=[-np.log10(0.05)] * df.log2FC.shape[0],
                             line=dict(color='red', dash='dash'),
                             name='p_value=0.05'))
    fig = go.Figure(data=traces, layout=go.Layout(title=title,
                                                  xaxis=dict(title='log2 fold change'),
                                                  yaxis=dict(title='-log10(pvalue)')))
    return fig

In [ ]:
old2young_tissue2volcano = {}
old_d2young_tissue2volcano = {}
old_d2old_tissue2volcano = {}

for tissue in ['plasma', 'liver', 'muscle', 'brain_cb', 'brain_pfc']:
    tmp_df = tissues_top[tissue]
    old2young_tissue2volcano[tissue] = calc_volcano(tmp_df[tmp_df.age == 'young'],
                                                    tmp_df[(tmp_df.age == 'old')&(tmp_df.vitamin == 'no')])
    old_d2young_tissue2volcano[tissue] = calc_volcano(tmp_df[(tmp_df.age == 'young')],
                                                     tmp_df[(tmp_df.age == 'old')&(tmp_df.vitamin == 'yes')])
    old_d2old_tissue2volcano[tissue] = calc_volcano(tmp_df[(tmp_df.age == 'old')&(tmp_df.vitamin == 'no')],
                                                    tmp_df[(tmp_df.age == 'old')&(tmp_df.vitamin == 'yes')])

In [ ]:
tissue = 'liver'
title = 'M/Z [{}]'.format(tissue)
max_val = np.max(list(old2young_tissue2volcano[tissue].minus_log10_pvalue.values) +
                 list(old_d2young_tissue2volcano[tissue].minus_log10_pvalue.values)) + 0.1
iplot(volcano_figures(old2young_tissue2volcano[tissue], old_d2young_tissue2volcano[tissue], title, max_val))

In [ ]:
tissue = 'plasma'
title = 'M/Z [{}]'.format(tissue)
max_val = np.max(list(old2young_tissue2volcano[tissue].minus_log10_pvalue.values) +
                 list(old_d2young_tissue2volcano[tissue].minus_log10_pvalue.values)) + 0.1
iplot(volcano_figures(old2young_tissue2volcano[tissue], old_d2young_tissue2volcano[tissue], title, max_val))

In [ ]:
tissue = 'muscle'
title = 'M/Z [{}]'.format(tissue)
max_val = np.max(list(old2young_tissue2volcano[tissue].minus_log10_pvalue.values) +
                 list(old_d2young_tissue2volcano[tissue].minus_log10_pvalue.values)) + 0.1
iplot(volcano_figures(old2young_tissue2volcano[tissue], old_d2young_tissue2volcano[tissue], title, max_val))

In [ ]:
tissue = 'brain_cb'
title = 'M/Z [{}]'.format(tissue)
max_val = np.max(list(old2young_tissue2volcano[tissue].minus_log10_pvalue.values) +
                 list(old_d2young_tissue2volcano[tissue].minus_log10_pvalue.values)) + 0.1
iplot(volcano_figures(old2young_tissue2volcano[tissue], old_d2young_tissue2volcano[tissue], title, max_val))

In [ ]:
tissue = 'brain_pfc'
title = 'M/Z [{}]'.format(tissue)
max_val = np.max(list(old2young_tissue2volcano[tissue].minus_log10_pvalue.values) +
                 list(old_d2young_tissue2volcano[tissue].minus_log10_pvalue.values)) + 0.1
iplot(volcano_figures(old2young_tissue2volcano[tissue], old_d2young_tissue2volcano[tissue], title, max_val))

#### 6. Draw PCA

In [ ]:
def get_pca_components_df(df_new):
    mz_columns = sorted(list(set(df_new.columns) - set(
    ['Unnamed: 0', 'tissue', 'age', 'rat_type', 'vitamin', 'sample', 'Tissue'])))
    X = df_new[mz_columns].copy()
    X = StandardScaler().fit_transform(X)
    pca = PCA(n_components=2)
    components = pca.fit_transform(X)
    pca_df = pd.DataFrame(data=components, columns=['pca_x', 'pca_y'])
    pca_df['target'] = df_new.tissue.values
    pca_df['rat_type'] = df_new.rat_type.values
    return pca_df

PCA for the different tissues of all rats

In [ ]:
pca_df = get_pca_components_df(norm_data)
fig = px.scatter(pca_df, x="pca_x", y="pca_y", color="target", title='Different tissues PCA')
fig.show()

PCA for every tissue for different rat groups

In [ ]:
tissue = 'liver'
pca_df = get_pca_components_df(norm_data[norm_data.tissue==tissue].copy())
fig = px.scatter(pca_df, x="pca_x", y="pca_y", color="rat_type", title=tissue)
fig.show()

In [ ]:
tissue = 'brain_cb'
pca_df = get_pca_components_df(norm_data[norm_data.tissue==tissue].copy())
fig = px.scatter(pca_df, x="pca_x", y="pca_y", color="rat_type", title=tissue)
fig.show()

In [ ]:
tissue = 'brain_pfc'
pca_df = get_pca_components_df(norm_data[norm_data.tissue==tissue].copy())
fig = px.scatter(pca_df, x="pca_x", y="pca_y", color="rat_type", title=tissue)
fig.show()

In [ ]:
tissue = 'muscle'
pca_df = get_pca_components_df(norm_data[norm_data.tissue==tissue].copy())
fig = px.scatter(pca_df, x="pca_x", y="pca_y", color="rat_type", title=tissue)
fig.show()

In [ ]:
tissue = 'plasma'
pca_df = get_pca_components_df(norm_data[norm_data.tissue==tissue].copy())
fig = px.scatter(pca_df, x="pca_x", y="pca_y", color="rat_type", title=tissue)
fig.show()